#### Prediction with Large Language Models

--- TODO 底下的code可以在final notebook 裡移除或放到最上面 baseline method應該也會有一樣的split data ----

In [1]:
from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


In [2]:
from huggingface_hub import login
login()  # Required for accessing gated models like Llama


In [24]:
%pip install -q --upgrade transformers datasets accelerate torch torchvision torchaudio

In [3]:
import pickle
import pandas as pd
import numpy as np
import os

pkl_path = "/content/drive/MyDrive/258 final project/processed_df.pkl"

with open(pkl_path, 'rb') as f:
    df = pickle.load(f)

print(f"Loaded dataframe with shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

df.head()


Loaded dataframe with shape: (4800, 32)
Columns: ['budget', 'genres', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'vote_average', 'vote_count', 'cast', 'crew', 'title', 'num_genres', 'num_keywords', 'num_production_companies', 'num_production_countries', 'num_spoken_languages', 'num_cast', 'num_crew', 'director', 'top_cast', 'genre_list', 'lm_input', 'lm_input_length']


,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,num_production_companies,num_production_countries,num_spoken_languages,num_cast,num_crew,director,top_cast,genre_list,lm_input,lm_input_length
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,4,2,2,83,153,[James Cameron],"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[Action, Adventure, Fantasy, Science Fiction]","Title: Avatar\nBudget: 237000000, revenue: 278...",226
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,3,1,1,34,32,[Gore Verbinski],"[Johnny Depp, Orlando Bloom, Keira Knightley]","[Adventure, Fantasy, Action]",Title: Pirates of the Caribbean: At World's En...,230
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",...,3,2,5,83,155,[Sam Mendes],"[Daniel Craig, Christoph Waltz, Léa Seydoux]","[Action, Adventure, Crime]","Title: Spectre\nBudget: 245000000, revenue: 88...",198
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,4,1,1,158,217,[Christopher Nolan],"[Christian Bale, Michael Caine, Gary Oldman]","[Action, Crime, Drama, Thriller]",Title: The Dark Knight Rises\nBudget: 25000000...,272
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,1,1,1,27,132,[Andrew Stanton],"[Taylor Kitsch, Lynn Collins, Samantha Morton]","[Action, Adventure, Science Fiction]","Title: John Carter\nBudget: 260000000, revenue...",241


--------------------------------------------------------------------

#### Data Splitting
We split data into 70% Train, 15% Validation, 15% Test

In [4]:
from sklearn.model_selection import train_test_split

X = df["lm_input"].astype(str).tolist()
y = df["vote_average"].astype(float).values

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=True
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True
)

print(f"\nData split:")
print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")



Data split:
Train: 3360, Val: 720, Test: 720


#### Definition of Dataset:
We define a PyTorch Dataset class that tokenizes movie text inputs and pairs them with rating labels. The dataset handles tokenization with a maximum sequence length of 512 tokens, padding, and truncation to ensure consistent input dimensions for the language models.

In [5]:
import torch
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

class MovieDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        if not text or text.strip() == "":
            text = "Empty text"

        enc = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt",
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item




#### Definition of Metrics:
We use RMSE (Root Mean Squared Error) and MAE (Mean Absolute Error) as evaluation metrics. RMSE penalizes larger errors more heavily, while MAE provides a more interpretable average error magnitude. Both metrics are computed on the test set to assess model performance.

In [6]:
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    # Handle tuple output (e.g., from T5)
    if isinstance(preds, tuple):
        preds = preds[0]

    # Flatten predictions
    preds = preds.reshape(-1)
    labels = labels.reshape(-1)

    # Handle NaN or inf values
    mask = np.isfinite(preds) & np.isfinite(labels)
    if mask.sum() == 0:
        return {"mse": float('inf'), "rmse": float('inf'), "mae": float('inf')}

    preds = preds[mask]
    labels = labels[mask]

    mse = mean_squared_error(labels, preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(labels, preds)

    return {"mse": mse, "rmse": rmse, "mae": mae}

#### Training:
We explore two approaches for using large language models to predict movie ratings.

**Method 1: Fine-tune LLM** directly adapts the pre-trained language model to the regression task by updating all model parameters during training. This approach allows the model to learn task-specific representations and can potentially achieve better performance, but requires more computational resources and training time.

**Method 2: Frozen LLM + Regressor** keeps the language model frozen and only trains a separate regressor on extracted features. This method is computationally efficient and faster to train, but may be limited by the fixed feature representations from the frozen model.

We evaluate three model architectures: DeBERTa-v3-base (encoder-only), Qwen2.5-0.5B (decoder-only), and Llama-3.2-1B-Instruct (decoder-only). Encoder-only models like DeBERTa are well-suited for understanding tasks and have shown strong performance on classification and regression. Decoder-only models like Qwen and Llama, while typically used for generation, can also be adapted for regression tasks. We expect encoder-only models to perform well due to their bidirectional understanding, while decoder-only models may offer different representational capabilities.

##### Method 1: Fine-tune LLM for Regression:
This method fine-tunes the entire language model on the movie rating prediction task. We add a regression head to the pre-trained model and update all parameters through backpropagation. The model learns to map movie descriptions directly to ratings, potentially capturing complex relationships between text features and ratings.

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TrainerCallback
import gc
import time
import torch

# Define a simple callback to print metrics nicely
class SimpleMetricsCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics:
            print(f"\n>> Epoch {state.epoch:.1f}: Val RMSE={metrics.get('eval_rmse', 'N/A'):.4f}, Val MAE={metrics.get('eval_mae', 'N/A'):.4f}\n")

def run_experiment(model_name):
    print(f"Training: {model_name}")

    device = "cuda" if torch.cuda.is_available() else "cpu"

    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, trust_remote_code=True)
    except:
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, trust_remote_code=True)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token if tokenizer.eos_token else tokenizer.unk_token

    train_ds = MovieDataset(X_train, y_train, tokenizer)
    val_ds = MovieDataset(X_val, y_val, tokenizer)
    test_ds = MovieDataset(X_test, y_test, tokenizer)

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=1, trust_remote_code=True, ignore_mismatched_sizes=True
    )

    model.config.problem_type = "regression"

    if hasattr(model.config, 'pad_token_id'):
        model.config.pad_token_id = tokenizer.pad_token_id

    # Training config
    batch_size = 2 if "llama" in model_name.lower() or "qwen" in model_name.lower() else 4
    args = TrainingArguments(
        output_dir=f"./results_{model_name.replace('/', '_').replace('-', '_')}",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        fp16=torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 7,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="rmse",
        greater_is_better=False,
        save_total_limit=1,
        report_to="none",
        logging_steps=50,
        logging_first_step=True,
        warmup_steps=100,
        max_grad_norm=1.0,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2), SimpleMetricsCallback()]
    )

    start_time = time.time()
    trainer.train()
    train_time = time.time() - start_time

    # Evaluate
    eval_results = trainer.evaluate(test_ds)

    # Cleanup
    del trainer, model, tokenizer
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return {
        "Method": "Method 1: Fine-tune",
        "Model": model_name,
        "RMSE": eval_results["eval_rmse"],
        "MAE": eval_results["eval_mae"],
        "Time(s)": train_time
    }

In [8]:
method1_results = []

models = [
    "meta-llama/Llama-3.2-1B-Instruct",
    "microsoft/deberta-v3-base",
    "Qwen/Qwen2.5-0.5B",

]

print("Method 1: Fine-tune LLM")
for model in models:
    try:
        res = run_experiment(model)
        method1_results.append(res)
        print(f"  Completed: {model} - RMSE: {res['RMSE']:.4f}, MAE: {res['MAE']:.4f}")
    except Exception as e:
        print(f"  Failed: {model} - {str(e)[:100]}")
        continue


Method 1: Fine-tune LLM
Training: meta-llama/Llama-3.2-1B-Instruct


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-0.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2676770795.py:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Mse,Rmse,Mae
1,5.445000,1.177156,1.177156,1.084968,0.723294
2,5.042300,1.079757,1.079757,1.039113,0.783121
3,1.779900,0.847020,0.847020,0.920337,0.620366



>> Epoch 1.0: Val RMSE=1.0850, Val MAE=0.7233


>> Epoch 2.0: Val RMSE=1.0391, Val MAE=0.7831


>> Epoch 3.0: Val RMSE=0.9203, Val MAE=0.6204




>> Epoch 3.0: Val RMSE=0.9564, Val MAE=0.6465

  Completed: Qwen/Qwen2.5-0.5B - RMSE: 0.9564, MAE: 0.6465



#### Method 2: Frozen LLM + Regressor:
This method extracts fixed feature representations from a frozen pre-trained language model, then trains a separate Ridge regressor on these features. The LLM parameters remain unchanged, acting as a feature extractor. This approach is faster and requires less memory, but the model cannot adapt its representations to the specific task.

In [9]:
# Feature extraction function
from transformers import AutoModel, AutoTokenizer
import torch

def extract_features(texts, llm, tokenizer, device='cuda', batch_size=16, use_cls=True):
    """
    Extract feature vectors from frozen LLM.

    Args:
        texts: List of texts
        llm: Frozen LLM model
        tokenizer: Tokenizer
        device: 'cuda' or 'cpu'
        batch_size: Batch size for feature extraction
        use_cls: True to use [CLS] token, False for mean pooling

    Returns:
        numpy array of shape [n_samples, hidden_dim]
    """
    llm.to(device)
    features = []

    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]

            encodings = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            ).to(device)

            outputs = llm(**encodings)
            hidden_states = outputs.last_hidden_state  # [batch_size, seq_len, hidden_dim]

            if use_cls:
                # Use [CLS] token (first token)
                batch_features = hidden_states[:, 0, :]
            else:
                # Mean pooling with attention mask
                attention_mask = encodings['attention_mask']
                mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size())
                sum_embeddings = torch.sum(hidden_states * mask_expanded, dim=1)
                sum_mask = torch.clamp(mask_expanded.sum(dim=1), min=1e-9)
                batch_features = sum_embeddings / sum_mask

            features.append(batch_features.cpu())

    return torch.cat(features, dim=0).numpy()



In [10]:
from sklearn.linear_model import Ridge
from transformers import AutoModel

def run_frozen_llm_experiment(model_name, regressor_type='ridge', batch_size=16):
    print(f"Training: {model_name} + {regressor_type}")

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    start_time = time.time()

    # Load and freeze LLM
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, trust_remote_code=True)
    except:
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, trust_remote_code=True)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token if tokenizer.eos_token else tokenizer.unk_token

    llm = AutoModel.from_pretrained(model_name, trust_remote_code=True)
    for param in llm.parameters():
        param.requires_grad = False
    llm.eval()

    # Extract features
    X_train_features = extract_features(X_train, llm, tokenizer, device, batch_size)
    X_test_features = extract_features(X_test, llm, tokenizer, device, batch_size)

    # Train regressor
    regressor = Ridge(alpha=1.0)
    regressor.fit(X_train_features, y_train)

    # Evaluate
    y_test_pred = regressor.predict(X_test_features)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)

    total_time = time.time() - start_time

    # Cleanup
    del llm, tokenizer, regressor
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return {
        "Method": "Method 2: Frozen LLM + Regressor",
        "Model": model_name,
        "RMSE": test_rmse,
        "MAE": test_mae,
        "Time(s)": total_time
    }


In [11]:
method2_results = []

models = [
    "microsoft/deberta-v3-base",
    "Qwen/Qwen2.5-0.5B",
    "meta-llama/Llama-3.2-1B-Instruct"
]

print("Method 2: Frozen LLM + Regressor")
for model in models:
    try:
        res = run_frozen_llm_experiment(model, regressor_type='ridge', batch_size=16)
        method2_results.append(res)
        print(f"  Completed: {model} - RMSE: {res['RMSE']:.4f}, MAE: {res['MAE']:.4f}")
    except Exception as e:
        print(f"  Failed: {model} - {str(e)[:100]}")
        continue


Method 2: Frozen LLM + Regressor
Training: microsoft/deberta-v3-base + ridge
  Completed: microsoft/deberta-v3-base - RMSE: 1.0078, MAE: 0.7237
Training: Qwen/Qwen2.5-0.5B + ridge
  Completed: Qwen/Qwen2.5-0.5B - RMSE: 1.1786, MAE: 0.8228
Training: meta-llama/Llama-3.2-1B-Instruct + ridge
  Completed: meta-llama/Llama-3.2-1B-Instruct - RMSE: 1.1786, MAE: 0.8228


#### Results Comparison:
We compare the performance of both methods across all models using RMSE and MAE metrics. The results are sorted by RMSE to identify the best-performing approach.

In [12]:
# Combine and display results
all_results = method1_results + method2_results

if len(all_results) > 0:
    df_results = pd.DataFrame(all_results)
    df_results = df_results[['Method', 'Model', 'RMSE', 'MAE', 'Time(s)']]
    df_results['Time(min)'] = df_results['Time(s)'] / 60
    df_results = df_results.drop('Time(s)', axis=1)
    df_results = df_results.sort_values('RMSE')

    print("\n=== Final Results ===")
    display(df_results)

    best = df_results.iloc[0]
    print(f"\nBest Model: {best['Method']} - {best['Model']}")
    print(f"  RMSE: {best['RMSE']:.4f}, MAE: {best['MAE']:.4f}, Time: {best['Time(min)']:.2f} min")
else:
    print("No results available.")



=== Final Results ===


,Method,Model,RMSE,MAE,Time(min)
0,Method 1: Fine-tune,meta-llama/Llama-3.2-1B-Instruct,0.937676,0.616214,10.847308
2,Method 1: Fine-tune,Qwen/Qwen2.5-0.5B,0.956361,0.646457,13.559481
3,Method 2: Frozen LLM + Regressor,microsoft/deberta-v3-base,1.007832,0.723742,0.471863
1,Method 1: Fine-tune,microsoft/deberta-v3-base,1.131605,0.834940,4.258394
4,Method 2: Frozen LLM + Regressor,Qwen/Qwen2.5-0.5B,1.178563,0.822759,1.241264
5,Method 2: Frozen LLM + Regressor,meta-llama/Llama-3.2-1B-Instruct,1.178565,0.822762,2.610020



Best Model: Method 1: Fine-tune - meta-llama/Llama-3.2-1B-Instruct
  RMSE: 0.9377, MAE: 0.6162, Time: 10.85 min
